# Proton propagation using CRPropa3

Gabriel de Azeredo, April 2024

Proton propagation through an expanding Universe, filled by CMB. Source at 500 Mpc.

Electron pair production and pion production is take into account as interaction with photon background.

This script generate earth's measured spectrum of a source emitting 1000 protons with 1e20 eV. 

In [1]:
#    Module import

from crpropa import *

In [2]:
#    Simulation general setup

events = 1000
source_distance = 500 * Mpc
energy_range = 1e2 * EeV

output_name = 'sim/04_sim1D_MonochromaticSource.txt'

#    Photon field setup

cmb = CMB()

In [3]:
#    Monochromatic source setup

position = SourcePosition(source_distance)
direction = SourceDirection(Vector3d(-1, 0, 0)) 
redshift = SourceRedshift1D() # Add a redshift
energySpectrum = SourceEnergy(energy_range)
particleType = SourceParticleType(nucleusId(1, 1)) # Emitting protons
source = Source()

source.add(position)
source.add(redshift) 
source.add(direction)
source.add(energySpectrum)
source.add(particleType)

In [4]:
#    Output file setup

output_type = Output.Trajectory1D     
output = TextOutput(output_name, output_type)

output.enable(Output.CurrentEnergyColumn)
output.enable(Output.RedshiftColumn)
output.enable(Output.CurrentPositionColumn)

#    Units setup

output.setEnergyScale(eV)
output.setLengthScale(Mpc)

#    Only protons

output.disable(Output.CreatedIdColumn)
output.disable(Output.CurrentIdColumn)

In [5]:
#    Observer setup

observer_type = Observer1D()

observer = Observer()
observer.add(observer_type)

In [6]:
#    Energy loss processes 

processes = []

#    Adiabatic energy loss

setCosmologyParameters(0.673, 1) 

z = Redshift()
processes.append(z)

#    Electron pair production

eppCMB = ElectronPairProduction(cmb)
processes.append(eppCMB)

#    Photo pion production

pppCMB = PhotoPionProduction(cmb)
processes.append(pppCMB)

In [7]:
#    Defining propagator

propagator = SimplePropagation(0.1 * kpc, 0.5 * Mpc)

In [8]:
#    Breaking conditions

max_trajectory = MaximumTrajectoryLength(500 * Mpc)

In [9]:
#    Assemble simulation modules

sim = ModuleList()

sim.add(output) # Output
sim.add(observer)  # Observer
[sim.add(process) for process in processes] # Energy loss 
sim.add(propagator) # Propagator
sim.add(max_trajectory) # Break conditions

sim.setShowProgress(True)

In [10]:
#    Simulation run

sim.run(source, events, True)
output.close()

crpropa::ModuleList: Number of Threads: 12
Run ModuleList
